In [9]:
## read in the adjlist, followers and its list of friends
import csv
import re
def cleanData(input_file):
    """
    : type input_file: list[str] 
    a text file name, each row is a string 
    starting with a screen_name, and a list of id_str's  of his fiends, separated by ','
    : rtype : list[list]
     remove some rows, with 0 friends or have 'NA' as their friends, which indicates 
                downloading unsucessfully
    """

    f = open(input_file, "r")
    adjlist = []
    failed_sn = []
    #removed those having 0 friends, having 'NA' as friends -- did not download friends list sucessfullly,
    for line in f:
        a = line.split(',')
        a =[re.sub("\n", "", x) for x in a]
        if len(a) >=3 or (len(a) == 2 and a[1] != 'NA'):
            adjlist.append(a)    #adjlist is a list[list]
        if len(a) ==2 and a[1] =='NA' :
            failed_sn.append(a[0])    
 
    return (adjlist, failed_sn) 
  
 
    

In [10]:
# random vs non-random
#first sample
"""
adjlist,failed = cleanData("../data/followers_Network/random100K.txt")
adjlist1, failed1 = cleanData("../data/followers_Network/non-random100K.txt")
print(len(adjlist),len(failed), len(adjlist1), len(failed1)) # 27,29 are having 0 friends
"""

adjlist,failed = cleanData("/p/stat/songwang/Bip100k2/random100K.txt")
adjlist1, failed1 = cleanData("/p/stat/songwang/Bip100k2/non-random100K.txt")
print(len(adjlist),len(failed), len(adjlist1), len(failed1)) # 27,29 are having 0 friends


23601 2621 26881 3115


In [11]:
#recreate  the sample100000.csv, since the origin file missing
def extraId_str (input_file):
    all_sn = []
    failed_sn = []
    f = open(input_file, "r")
    for line in f:
        items = line.split(',')
        items = [re.sub("\n|\t", "", x) for x in items]
        all_sn.append(items[0])
        if len(items) ==2 and items[1] =='NA' :
            failed_sn.append(items[0])
    return (all_sn, failed_sn)        

"""
all1, failed1 = extraId_str("../data/followers_Network/random100K.txt")
all2, failed2 = extraId_str("../data/followers_Network/non-random100K.txt")
print (len(all1), len(failed1), len(all2), len(failed2))
"""
all1, failed1 = extraId_str("/p/stat/songwang/Bip100k2/random100K.txt")
all2, failed2 = extraId_str("/p/stat/songwang/Bip100k2/non-random100K.txt")
print (len(all1), len(failed1), len(all2), len(failed2))


26241 2621 30011 3115


In [12]:
import csv
sn2id_str = {}
with open('../data/followers_Network/id_counts.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        sn = row['screen_name']
        id_str = row['id_str'] 
        if not sn in sn2id_str:
            sn2id_str[sn] = id_str
print(len(sn2id_str))
#3 fewer than number of rows  5138806
idAll1 =[sn2id_str[sn] if sn in sn2id_str else 'NA' for sn in all1]
idAll2 =[sn2id_str[sn] if sn in sn2id_str else 'NA' for sn in all2]
print(len(idAll1),len(idAll2))

5138803
26241 30011


In [13]:
nNA =0 
for i in idAll1:
    if i =='NA':
        nNA += 1
        print(i)
print(nNA) 
nNA =0 
for i in idAll2:
    if i =='NA':
        nNA += 1
        print(i)
print(nNA) 

import csv
import pandas as pd
data = {'random' : pd.Series(idAll1),  'non-random': pd.Series(idAll2)}
df = pd.DataFrame(data)
#df.to_csv('../data/followers_Network/sample100000_recovered.csv')


0
0


In [14]:
#exploring the reasons for failures


#1 # proteced rate 
import csv
protected = {}
with open('../data/followers_Network/id_counts.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        #print (row)
        sn = row['screen_name']
        status = row['protected']
        if not sn in protected:
            protected[sn] = status
            
  

len(protected.keys())
print(protected['aok_89'])

True


In [15]:
## overall rate:
count =0 

for sn in protected.keys():
    if protected[sn] == 'True':
        count +=1
        
print(len(protected), count, count/len(protected))


protected1 = []
count1 =0
for sn in failed1:
    if protected[sn] == 'True':
        count1 += 1
    protected1.append(protected[sn])
print(len(failed1), count1, count1/len(failed1))        

count2 = 0
protected2 = []
for sn in failed2:
    if protected[sn] == 'True':
        count2 += 1
    protected2.append(protected[sn])
print(len(failed2), count2, count2/len(failed2))           

5138803 420123 0.08175503127868494
2621 2027 0.7733689431514689
3115 2415 0.7752808988764045


In [16]:
print(failed1[:20], failed2[:20], protected1[:20], protected2[:20])
# reasons for failure :
"""
1, Account suspended -- most cases
2, Doesn't exist  -- 
3, protected now
4, some are fine, why? 'soutientMarina'
"""
print(len(protected2))
i = 0
c =0
while i <len(protected2):
    if protected2[i] != 'True':
        c +=1
        print(failed2[i], protected2[i])
        if c >10:
            break
    i += 1    

['Harrell6Myra', 'larry_drozda', 'Raabitshack', 'justinbilderbac', 'Lewshuigi', 'shangkumi', 'Bigberg25', 'ObrechtPaul', 'GddssFrstClss', 'tonyuck', 'NomanRiasat1', 'chloepiperrr', 'ippilearning', '601baronne', 'awalzie', 'fcordero11', 'bevhillstrainer', 'HeatherW1192', 'yancarras3', 'OzzySr'] ['isabelReynosoC', 'yzznxx25', 'nathan_Ingram', 'laura_mickey', 'InFuZioNclanEU', 'dannycaceres024', 'bradmorrical', 'ejps3', 'TimKelley5', 'jbrennz620', 'Calluev', 'SeldonPlan1000', 'Nyashaddy', 'danny_jacks0n', 'NolonaChaney', 'llcoopll', 'mathavankandiah', 'KitWyatt', 'Zla_Zla_Vestica', 'Hambone_61'] ['True', 'True', 'True', 'True', 'True', 'False', 'True', 'True', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True'] ['False', 'False', 'True', 'True', 'False', 'True', 'False', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True']
3115
isabelReynosoC False
yzznxx25 False
InFuZioNclanEU False
bradmorri

In [17]:
n_True = 0
n_total = 0
with open('../data/followers_Network/id_counts.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        n_total += 1
        if row['protected'] == 'True':
            n_True += 1
print(n_True, n_total, n_True/n_total)

424124 5188806 0.08173826502667474


In [18]:
class createBip(object):
    
    def __init__(self, adjlist, threshold):
        """
        adjlist type: list[list] first is follower_id, the rest are the friendsID
        rtype : None
        """
        self.adjlist = adjlist
        self.threshold = threshold
        self.edgelist = []
        self.reducedAdjlist = []
        
    
    def calculateFriendsCount(self):
        """
        keep the friends with >= threshold followers
        """
        friendsCount = {}
        for line in self.adjlist:
            for id_str in line[1:]:
                if id_str in friendsCount.keys():
                    friendsCount[id_str] = friendsCount[id_str] +1
                else:
                    friendsCount[id_str] = 1
        friendsCount_threshold = {}           
        for key in friendsCount.keys():
            if friendsCount[key] >= self.threshold:
                friendsCount_threshold[key] = friendsCount[key]
        #for key in friendsCount.keys(): 
        #    if friendsCount[key] < threshold:
        #        del friendsCount[key]  # size changed during the iteration
        return friendsCount_threshold        
                
    def createEdgelist(self):
        """
        create the edgelist from the adjlist
        """
        friendsCount = self.calculateFriendsCount()
        #newAdjlist = []
        for line in self.adjlist:
            a = []
            a.append(line[0])
            for id_str in line[1:]:
                if id_str in friendsCount.keys():
                    a.append(id_str)
                    self.edgelist.append( (line[0],id_str) )
            self.reducedAdjlist.append(a) 
        
                    



In [19]:
import csv
#random graph

BipGraph = createBip(adjlist,5)  #2
BipGraph.createEdgelist()
"""
'../data/followers_Network/edgelist-random100K-2.csv'
"""

with open('/p/stat/songwang/Bip100k2/edgelist-random100K-5.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['follower_id_str','friends_id_str'])
    for row in BipGraph.edgelist:
        csv_out.writerow(row)
        
fcount =BipGraph.calculateFriendsCount()
#print(fcount)
"""
'../data/followers_Network/friendsIDs-random100K.csv'
"""
with open('/p/stat/songwang/Bip100k2/friendsIDs-random100K.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['friends_id_str', 'followerCount'])
    i = 0
    for key in fcount.keys():
        csv_out.writerow((i, key,fcount[key]))
        i += 1

In [20]:

import csv
#non-random graph

BipGraph1 = createBip(adjlist1, 5)
BipGraph1.createEdgelist()
"""
'../data/followers_Network/edgelist-non-random100K-2.csv'
"""
with open('/p/stat/songwang/Bip100k2/edgelist-non-random100K-5.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['follower_id_str','friends_id_str'])
    for row in BipGraph1.edgelist:
        csv_out.writerow(row)
        
        
fcount1 =BipGraph1.calculateFriendsCount()
print( len(fcount1))    
"""
'../data/followers_Network/friendsIDs-non-random100K.csv'
"""
with open('/p/stat/songwang/Bip100k2/friendsIDs-non-random100K.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['friends_id_str', 'followerCount'])
    i = 0
    for key in fcount1.keys():
        csv_out.writerow((i, key,fcount1[key]))   
        i += 1

139675


In [21]:

# 5 threshold, friends count
print(len(fcount), len(fcount1))
#214821 374004
# 2 shrehold
#831808 1474288

118239 139675


In [22]:
## explore degrees, including the person self. deg ==2, just one friends
length = [len(x) for x in adjlist1] # row degee, including 
table_length = {}
for x in length:
    if x in table_length:
        table_length[x] += 1
    else:
        table_length[x] = 1
        

#print(table_length)        # not following #54450 contailing one friends, are they NA?

ones = []
for x in adjlist1: #adjlist
    if len(x) ==2:
        #print(x)
        ones.append(x)

print(len(ones))

len(adjlist[0])

184


69